In [1]:
class Add:
    def __call__(self, x, y):
        return x + y

In [2]:
add = Add()
print(add(1, 2))

3


In [3]:
print(type(add))

<class '__main__.Add'>


In [4]:
def add_func(x, y):
    return x + y

print(type(add_func))

<class 'function'>


In [5]:
print(add.__call__(1, 2))

3


In [6]:
class Mul:
    def mul(self, x, y):
        return x * y

In [7]:
mul = Mul()
print(mul.mul(1, 2))
print(mul.mul.__call__(1, 2))

2
2


In [8]:
from datetime import datetime

def create_show_time(func):
    def show_time(*args, **kwargs):
        starttime = datetime.now()
        retval = func(*args, **kwargs)
        endtime = datetime.now()
        print(endtime - starttime)
        return retval
    
    return show_time

In [9]:
class Showtime:
    def __init__(self, func):
        self.func = func
        
    def __call__(self, *args, **kwargs):
        starttime = datetime.now()
        retval = self.func(*args, **kwargs)
        endtime = datetime.now()
        print(endtime - starttime)
        return retval

In [10]:
@Showtime
def add_1(x):
    return x + 1

In [11]:
print(add_1(1))

0:00:00
2


In [12]:
class XY:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def printx(self):
        print(self.x)

    def printy(self):
        print(self.y)

In [13]:
xy = XY(1, 2)
xy.printx()
xy.printy()

1
2


In [14]:
def create_xy(x, y):
    def printx():
        print(x)

    def printy():
        print(y)

    return { "printx": printx, "printy": printy }

In [15]:
xy2 = create_xy(1, 2)
xy2["printx"]()
xy2["printy"]()

1
2


In [16]:
class Mul:
    @create_show_time
    def mul(self, x, y):
        return x * y
    

In [17]:
mul = Mul()
print(mul.mul(1, 2))

0:00:00
2


In [18]:
class Mul:
    @Showtime
    def mul(self, x, y):
        return x * y
    
mul = Mul()
print(mul.mul(1, 2))

TypeError: Mul.mul() missing 1 required positional argument: 'y'

In [19]:
from ai import dprint
from functools import wraps
from copy import deepcopy
from random import choice

def ai_by_score(eval_func):
    @wraps(eval_func)
    def wrapper(mb_orig, debug=False, rand=True, analyze=False):
        dprint(debug, "Start ai_by_score")
        dprint(debug, mb_orig)
        legal_moves = mb_orig.calc_legal_moves()
        dprint(debug, "legal_moves", legal_moves)
        best_score = float("-inf")
        best_moves = []
        if analyze:
            score_by_move = {}
        for move in legal_moves:
            dprint(debug, "=" * 20)
            dprint(debug, "move", move)
            mb = deepcopy(mb_orig)
            x, y = move
            mb.move(x, y)
            dprint(debug, mb)
            
            score = eval_func(mb)
            dprint(debug, "score", score, "best score", best_score)
            if analyze:
                score_by_move[move] = score
            
            if best_score < score:
                best_score = score
                best_moves = [move]
                dprint(debug, "UPDATE")
                dprint(debug, "  best score", best_score)
                dprint(debug, "  best moves", best_moves)
            elif best_score == score:
                best_moves.append(move)
                dprint(debug, "APPEND")
                dprint(debug, "  best moves", best_moves)

        dprint(debug, "=" * 20)
        dprint(debug, "Finished")
        dprint(debug, "best score", best_score)
        dprint(debug, "best moves", best_moves)
        if analyze:
            return {
                "candidate": best_moves,
                "score_by_move": score_by_move,
            }
        elif rand:   
            return choice(best_moves)
        else:
            return best_moves[0]
        
    return wrapper

In [20]:
@ai_by_score
def ai2s(mb):
    return 0

In [21]:
from marubatsu import Marubatsu

mb = Marubatsu()
print(ai2s(mb))

(2, 0)


In [22]:
print(ai2s(mb, debug=True))

Start ai_by_score
Turn o
...
...
...

legal_moves [(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)]
move (0, 0)
Turn x
O..
...
...

score 0 best score -inf
UPDATE
  best score 0
  best moves [(0, 0)]
move (1, 0)
Turn x
.O.
...
...

score 0 best score 0
APPEND
  best moves [(0, 0), (1, 0)]
move (2, 0)
Turn x
..O
...
...

score 0 best score 0
APPEND
  best moves [(0, 0), (1, 0), (2, 0)]
move (0, 1)
Turn x
...
O..
...

score 0 best score 0
APPEND
  best moves [(0, 0), (1, 0), (2, 0), (0, 1)]
move (1, 1)
Turn x
...
.O.
...

score 0 best score 0
APPEND
  best moves [(0, 0), (1, 0), (2, 0), (0, 1), (1, 1)]
move (2, 1)
Turn x
...
..O
...

score 0 best score 0
APPEND
  best moves [(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1)]
move (0, 2)
Turn x
...
...
O..

score 0 best score 0
APPEND
  best moves [(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2)]
move (1, 2)
Turn x
...
...
.O.

score 0 best score 0
APPEND
  best moves [(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2,

In [23]:
print(ai2s(mb, rand=False))

(0, 0)


In [24]:
from pprint import pprint

pprint(ai2s(mb, analyze=True))

{'candidate': [(0, 0),
               (1, 0),
               (2, 0),
               (0, 1),
               (1, 1),
               (2, 1),
               (0, 2),
               (1, 2),
               (2, 2)],
 'score_by_move': {(0, 0): 0,
                   (0, 1): 0,
                   (0, 2): 0,
                   (1, 0): 0,
                   (1, 1): 0,
                   (1, 2): 0,
                   (2, 0): 0,
                   (2, 1): 0,
                   (2, 2): 0}}
